In [24]:
# 讀取資料並加上年份欄位
demo = pd.read_sas("../data/raw/P_DEMO.XPT", format="xport")
demo["year_group"] = "2017-2020"

lab = pd.read_sas("../data/raw/P_BIOPRO.xpt", format="xport")
lab["year_group"] = "2017-2020"

urine = pd.read_sas("../data/raw/P_ALB_CR.xpt", format="xport")
urine["year_group"] = "2017-2020"

demo2 = pd.read_sas("../data/raw/DEMO_I.XPT", format="xport")
demo2["year_group"] = "2015-2016"

lab2 = pd.read_sas("../data/raw/BIOPRO_I.xpt", format="xport")
lab2["year_group"] = "2015-2016"

urine2 = pd.read_sas("../data/raw/ALB_CR_I.xpt", format="xport")
urine2["year_group"] = "2015-2016"

demo3 = pd.read_sas("../data/raw/DEMO_H.XPT", format="xport")
demo3["year_group"] = "2013-2014"

lab3 = pd.read_sas("../data/raw/BIOPRO_H.xpt", format="xport")
lab3["year_group"] = "2013-2014"

urine3 = pd.read_sas("../data/raw/ALB_CR_H.xpt", format="xport")
urine3["year_group"] = "2013-2014"


common_demo_cols = list(set(demo.columns) & set(demo2.columns))
common_lab_cols = list(set(lab.columns) & set(lab2.columns))
common_urine_cols = list(set(urine.columns) & set(urine3.columns))
# urine 欄位已完全一致

demo_all = pd.concat([demo[common_demo_cols], demo2[common_demo_cols], demo3[common_demo_cols]], ignore_index=True)
lab_all = pd.concat([lab[common_lab_cols], lab2[common_lab_cols], lab3[common_lab_cols]], ignore_index=True)
urine_all = pd.concat([urine[common_urine_cols], urine2[common_urine_cols], urine3[common_urine_cols]], ignore_index=True)



df = demo_all.merge(lab_all, on=["SEQN", "year_group"], how="inner")\
         .merge(urine_all, on=["SEQN", "year_group"], how="inner")
         

print(f"合併後資料共有 {df.shape[0]} 筆資料與 {df.shape[1]} 個欄位")

df["ACR"] = (df["URXUMA"] * 0.1) / df["URXUCR"] * 1000



# 建立 CKD 標籤（ACR ≥ 10）


df["ckd"] = df["ACR"].apply(lambda x: 1 if x > 20 else 0)

print(df["ckd"].value_counts(dropna=False))

合併後資料共有 24132 筆資料與 68 個欄位
ckd
0    19822
1     4310
Name: count, dtype: int64


In [25]:
# 列出缺失值數量（由多至少排序）
missing = df.isna().sum().sort_values(ascending=False)

# 篩出有缺值的欄位
missing = missing[missing > 0]
print(missing)

df_cleaned = df.dropna(axis=1, how="all")
missing_after_drop = df_cleaned.isna().sum()
print(missing_after_drop)
missing_ratio = (df_cleaned.isna().sum() / len(df_cleaned)).sort_values(ascending=False)
print(missing_ratio[missing_ratio > 0])
high_missing_cols = missing_ratio[missing_ratio > 0.5].index.tolist()

# 丟掉這些欄位
df_cleaned = df_cleaned.drop(columns=high_missing_cols, errors="ignore")

# 列出仍有缺值的欄位
missing_after_drop = df_cleaned.isna().sum()
cols_with_missing = missing_after_drop[missing_after_drop > 0].index.tolist()

# 對剩下有缺值的欄位：
# 數值欄位 → 補中位數
# 類別欄位 → 補眾數
for col in cols_with_missing:
    if df_cleaned[col].dtype in ["float64", "int64"]:
        df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].median())
    else:
        df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].mode(dropna=True)[0])


# 移除資訊洩漏欄位（分別為 ACR 的分子與分母）
df_cleaned = df_cleaned.drop(columns=["ACR", "URXUMS", "URXCRS", "SEQN", "URXUMA", "URXUCR", "URDACT"])

# 6. ✅ 下採樣 CKD = 0，使 CKD = 0 與 CKD = 1 比例為 2:1
df_ckd = df_cleaned[df_cleaned["ckd"] == 1]
df_non_ckd = df_cleaned[df_cleaned["ckd"] == 0].sample(n=int(1.5 * len(df_ckd)), random_state=42)

df_balanced = pd.concat([df_ckd, df_non_ckd]).sample(frac=1, random_state=42)


# 7. 儲存資料
print(f"平衡後資料共有 {df_balanced.shape[0]} 筆資料")
print(df_balanced["ckd"].value_counts())



df_balanced.to_csv("../data/processed/nhanes_2013_2020_ckd_cleaned.csv", index=False)

RIDAGEMN    24132
RIDEXPRG    19880
AIALANGA     5146
DMDEDUC2     4526
INDFMPIR     2636
LBXSLDSI     2062
LBDSCASI     1891
LBXSCA       1891
LBXSASSI     1890
LBDSIRSI     1879
LBXSIR       1879
LBXSCK       1873
LBDSGBSI     1863
LBXSGB       1863
LBDSTPSI     1862
LBXSTP       1862
LBXSKSI      1857
LBDSTRSI     1856
LBXSTR       1856
LBDSTBSI     1855
LBDSUASI     1854
LBXSTB       1854
LBXSUA       1854
LBDSCHSI     1852
LBXSOSSI     1852
LBXSATSI     1852
LBXSCH       1852
LBDSPHSI     1851
LBDSBUSI     1851
LBXSPH       1851
LBXSGTSI     1851
LBXSBU       1851
LBXSAPSI     1851
LBDSGLSI     1849
LBXSGL       1849
LBXSCR       1849
LBDSCRSI     1849
LBXSC3SI     1849
LBXSNASI     1846
LBXSAL       1846
LBXSCLSI     1846
LBDSALSI     1846
MIALANG      1675
MIAPROXY     1673
MIAINTRP     1672
FIAPROXY      929
FIAINTRP      929
FIALANG       929
URDACT        617
URXCRS        617
URXUCR        617
ACR           617
URXUMA        616
URXUMS        616
dtype: int64
SDMVSTRA       